In [1]:
import dlc_bci as bci
import torch
from torch.autograd import Variable
from torch import nn
from torch import optim
from torch import Tensor
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load the data

In [13]:
train_input , train_target = bci.load ( root = "./ data_bci")
test_input , test_target = bci.load ( root = "./ data_bci " , train = False )

train_target = Variable(train_target)
train_input = Variable(train_input.unsqueeze(dim=2))
test_input = Variable(test_input.unsqueeze(dim=2))
test_target = Variable(test_target)

print('train_input size:' , train_input.size())
print ( 'train_target size:' , train_target.size())
print ('test_input size:' , test_input.size())
print ('test_target size:' , test_target.size())

train_input size: torch.Size([316, 28, 1, 50])
train_target size: torch.Size([316])
test_input size: torch.Size([100, 28, 1, 50])
test_target size: torch.Size([100])


## Convolutional neural network

In [78]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d (28, 32, kernel_size = (1,5))
        self.avg_pool1 = nn.AvgPool2d(kernel_size = (1,4))
        self.conv2 = nn.Conv2d (32 , 64, kernel_size = (1,5))
        self.fc1 = nn.Linear (448 , 200)
        self.fc2 = nn.Linear (200 , 1)
        
    def forward (self , x):
        x = self.conv1(x)
        print('size after conv1 ',x.size())
        x = F.relu(self.avg_pool1(x))
        print('size after relu1 ',x.size())
        x = F.relu(self.conv2(x))
        print('size after relu2 ',x.size())
        x = x.view (-1, 448)
        x = F.relu ( self.fc1 (x))
        x = self.fc2(x)
                   #x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride =3) )
        #x = F.relu (F.max_pool2d ( self.conv2(x), kernel_size =2, stride =2) )
        return x

In [74]:
net = Net()

In [79]:
### train the model
epoch = 100
batch_size = 20
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.000001,momentum = 0.9)
i = 0
for e in range(epoch):
    for b in range(0,train_input.size(0),batch_size):
        input_ = train_input.narrow(0,b,min(batch_size,train_input.size(0)-b))
        target = train_target.narrow(0,b,min(batch_size,train_input.size(0)-b))
        output = net(input_)#.view(min(batch_size,train_input.size(0)-b),2)
        loss = criterion(output,target)
        net.zero_grad()
        loss.backward()
        optimizer.step()
    if e%100 == 0: # print the loss every 10 epochs
        output = net(test_2).view(-1,2)
        a,predicted_class = output.max(dim=1)
        predicted_class = predicted_class.view(-1,28)
        predicted_class = 1*(predicted_class.float().mean(dim=1) > 0.5)
        print(criterion(output,test_target_2).data[0],accuracy(predicted_class,test_target).data[0])

torch.Size([20, 32, 1, 46])
torch.Size([20, 32, 1, 11])
torch.Size([20, 64, 1, 7])


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /Users/soumith/minicondabuild3/conda-bld/pytorch_1512380529591/work/torch/lib/THNN/generic/ClassNLLCriterion.c:87

In [68]:
64*7

448